In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth  # Do this in local & cloud setups
else:
    import torch; v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [19]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.is_available()

True

In [4]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"   # Recommended for production quality
DATA_PATH = "/content/drive/MyDrive/spider_sft/train_sft.jsonl"
OUTPUT_DIR = "outputs/qwen-text2sql"

MAX_SEQ_LENGTH = 2048
BATCH_SIZE = 2
GRAD_ACCUM = 8
LR = 2e-4
EPOCHS = 2

In [5]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    dtype = None,
    # token = "YOUR_HF_TOKEN",      # HF Token for gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files=DATA_PATH,
    split="train"
)

dataset


Dataset({
    features: ['instruction', 'output'],
    num_rows: 7000
})

In [8]:
def format_example(example):
    text = f"""<|im_start|>system
You are a Text-to-SQL expert.<|im_end|>
<|im_start|>user
{example['instruction']}<|im_end|>
<|im_start|>assistant
{example['output']}<|im_end|>"""
    return {"text": text}

dataset = dataset.map(format_example)


In [9]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    warmup_steps = 50,
    num_train_epochs = EPOCHS,
    learning_rate = LR,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",
    seed = 42,
    output_dir = OUTPUT_DIR,
    save_steps = 500,
    save_total_limit = 2,
    report_to = "none",
)


In [10]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    args = training_args,
    packing = True,   # important for efficiency
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/7000 [00:00<?, ? examples/s]

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 2 | Total steps = 876
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 36,929,536 of 1,580,643,840 (2.34% trained)


Step,Training Loss
10,1.877000
20,1.351400
30,0.974800
40,0.889000
50,0.812700
60,0.704200
70,0.637000
80,0.554500
90,0.515400
100,0.400700


TrainOutput(global_step=876, training_loss=0.2691553580026104, metrics={'train_runtime': 2094.6309, 'train_samples_per_second': 6.684, 'train_steps_per_second': 0.418, 'total_flos': 3.1139918299201536e+16, 'train_loss': 0.2691553580026104, 'epoch': 2.0})

In [20]:
trainer.model.save_pretrained(f"/content/drive/MyDrive/{OUTPUT_DIR}")
tokenizer.save_pretrained(f"/content/drive/MyDrive/{OUTPUT_DIR}")


('/content/drive/MyDrive/outputs/qwen-text2sql/tokenizer_config.json',
 '/content/drive/MyDrive/outputs/qwen-text2sql/special_tokens_map.json',
 '/content/drive/MyDrive/outputs/qwen-text2sql/chat_template.jinja',
 '/content/drive/MyDrive/outputs/qwen-text2sql/vocab.json',
 '/content/drive/MyDrive/outputs/qwen-text2sql/merges.txt',
 '/content/drive/MyDrive/outputs/qwen-text2sql/added_tokens.json',
 '/content/drive/MyDrive/outputs/qwen-text2sql/tokenizer.json')

In [13]:
from datasets import load_dataset

dataset = load_dataset("xlangai/spider")

val_data = dataset["validation"]


README.md: 0.00B [00:00, ?B/s]

spider/train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

spider/validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})

In [21]:
def build_prompt(question, schema_text):
    return f"""You are a SQL expert.

Database schema:
{schema_text}

Question: {question}
SQL:"""


In [23]:
from transformers import  AutoModelForCausalLM
sft_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/outputs/qwen-text2sql")

In [ ]:
def generate_sql(model, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.0,
            do_sample=False
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text.split("SQL:")[-1].strip()
